# Training

In [3]:
from google.colab import drive
drive._mount('/content/drive')

Mounted at /content/drive


In [5]:
import os
os.chdir('/content/drive/MyDrive/LearningToCountEverything_new')

In [6]:
import torch.nn as nn
from model import *
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [7]:
enable_gpu = True
if enable_gpu:
  print("=====> GPU")
else:
  print("=====> CPU")

=====> GPU


In [8]:
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
image_path = '/content/drive/MyDrive/Colab Notebooks/LearningToCountEverything/data/'
output_dir = "./logsSave"
check_dir = "./checkpoints"
test_split = "train" #choices=["train", "test", "val"]
gpu_id = 0 
learning_rate = 1e-5
anno_file = image_path + 'annotation_FSC147_384.json'
data_split_file = image_path + 'Train_Test_Val_FSC_147.json'
im_dir = image_path + 'images_384_VarV2'
gt_dir = image_path + 'gt_density_map_adaptive_384_VarV2'


regressor = CountRegressorAML(6, pool='mean')
weights_normal_init(regressor, dev=0.001)

backbone_model = Resnet50FPN()

if enable_gpu: 
    regressor.cuda()
    regressor.train()
    criterion = nn.MSELoss().cuda()
    backbone_model.cuda()
    backbone_model.eval()
    os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
    os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu_id)
else: 
    regressor.train()
    criterion = nn.MSELoss()
    backbone_model.eval()

optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

if not exists(output_dir):
    os.mkdir(output_dir)

if not exists(check_dir):
    os.mkdir(check_dir)


  

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


  0%|          | 0.00/97.8M [00:00<?, ?B/s]

In [9]:
with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [10]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip


requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...


YOLOv5 🚀 2021-12-13 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)



  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect



Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [11]:
import warnings
warnings.filterwarnings("ignore")

In [12]:
len(data_split['train'])

3659

In [13]:
len(data_split['val'])

1286

In [ ]:
%matplotlib inline
run_train_phase(epochs=100, backbone_model=backbone_model, regressor=regressor, yolo_model=model_yolo,
                optimizer=optimizer, criterion=criterion, annotations=annotations, 
                data_train=data_split['train'], shuffle=True, data_val=data_split['val'], 
                num_img_train=len(data_split['train']), num_img_val=len(data_split['val']), 
                yolo_flag=True, yolo_threshold=3, 
                plot_flag=False, 
                save='/content/drive/MyDrive/LearningToCountEverything_new/data/pretainedModelsOurs/model_resnet_yolo.pth', 
                im_dir=im_dir, gt_dir=gt_dir,
                enable_gpu=enable_gpu, augment=True, 
                checkpoint_dir='checkpoints/', 
                load_checkpoint='/content/drive/MyDrive/LearningToCountEverything_new/checkpoints/checkpoint.pt')

Training on FSC147 train set data


actual:  91.0 -- predicted:  39.9 -- YOLO:   0.0 -- FAMNet error:  51.1 -- YOLO error:  91.0 -- Current MAE:51.13 -- RMSE:51.13 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   0%|          | 1/3659 [00:52<53:50:59, 53.00s/it]

actual:  72.0 -- predicted:   0.0 -- YOLO:   3.0 -- FAMNet error:  72.0 -- YOLO error:  69.0 -- Current MAE:61.56 -- RMSE:62.44 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   0%|          | 2/3659 [00:54<22:50:21, 22.48s/it]

actual:  14.0 -- predicted:  52.1 -- YOLO:  10.0 -- FAMNet error:  38.1 -- YOLO error:   4.0 -- Current MAE:53.75 -- RMSE:55.53 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   0%|          | 3/3659 [00:55<12:58:21, 12.77s/it]

actual:  35.0 -- predicted:  46.1 -- YOLO:  22.0 -- FAMNet error:  11.1 -- YOLO error:  13.0 -- Current MAE:43.09 -- RMSE:48.41 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   0%|          | 4/3659 [00:56<8:17:17,  8.16s/it] 

...

actual:  24.0 -- predicted:  57.1 -- YOLO:   1.0 -- FAMNet error:  33.1 -- YOLO error:  23.0 -- Current MAE:44.73 -- RMSE:71.53 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   2%|▏         | 76/3659 [02:04<57:29,  1.04it/s]  

actual:  20.0 -- predicted:  45.6 -- YOLO:  12.0 -- FAMNet error:  25.6 -- YOLO error:   8.0 -- Current MAE:44.48 -- RMSE:71.12 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   2%|▏         | 77/3659 [02:05<58:29,  1.02it/s]

actual:  10.0 -- predicted:  48.5 -- YOLO:   7.0 -- FAMNet error:  38.5 -- YOLO error:   3.0 -- Current MAE:44.40 -- RMSE:70.80 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   2%|▏         | 78/3659 [02:06<59:47,  1.00s/it]

actual:  23.0 -- predicted:  47.9 -- YOLO:   0.0 -- FAMNet error:  24.9 -- YOLO error:  23.0 -- Current MAE:44.16 -- RMSE:70.41 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   2%|▏         | 79/3659 [02:07<56:42,  1.05it/s]

actual:  28.0 -- predicted: 106.3 -- YOLO:  14.0 -- FAMNet error:  78.3 -- YOLO error:  14.0 -- Current MAE:44.58 -- RMSE:70.51 -- Best VAL MAE:10000000.00 -- RMSE: 10000000.00:   2%|▏         | 80/3659 [02:08<1:01:39,  1.03s/it]

# Testing

In [ ]:
import torch.nn as nn
from model import Resnet50FPN, CountRegressor, weights_normal_init, weights_xavier_init
from utils_ltce import MAPS, Scales, Transform,TransformTrain,extract_features, visualize_output_and_save
from PIL import Image
import os
import torch
import argparse
import json
import numpy as np
from tqdm import tqdm
from os.path import exists,join
import random
import torch.optim as optim
import torch.nn.functional as F
import cv2
import matplotlib.pyplot as plt
from collections import defaultdict
import torchvision.ops.boxes as bops
from funcs import *

In [ ]:
data_path = '/content/drive/MyDrive/LearningToCountEverything_new/data/'
output_dir = "./logsSave"
test_split = "test" #choices=["train", "test", "val"]
gpu = 0 
learning_rate = 1e-5
anno_file = data_path + 'annotation_FSC147_384.json'
data_split_file = data_path + 'Train_Test_Val_FSC_147.json'
im_dir = data_path + 'images_384_VarV2'
gt_dir = data_path + 'gt_density_map_adaptive_384_VarV2'


regressor = CountRegressor(6, pool='mean')

backbone_model = Resnet50FPN()

if enable_gpu: 

  regressor.cuda()
  regressor.eval()
  criterion = nn.MSELoss().cuda()
  backbone_model.cuda()
  backbone_model.eval()

else: 
  regressor.train()
  criterion = nn.MSELoss()
  backbone_model.eval()

optimizer = optim.Adam(regressor.parameters(), lr = learning_rate)

if not exists(output_dir):
    os.mkdir(output_dir)

os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = str(gpu)

In [ ]:
with open(anno_file) as f:
    annotations = json.load(f)

with open(data_split_file) as f:
    data_split = json.load(f)

In [ ]:
model_yolo = torch.hub.load('ultralytics/yolov5', 'yolov5l6', pretrained=True)

Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2021-12-12 torch 1.10.0+cu111 CUDA:0 (Tesla K80, 11441MiB)

Fusing layers... 
Model Summary: 476 layers, 76726332 parameters, 0 gradients
Adding AutoShape... 


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%%time
%matplotlib inline
test(data_split['test'][100:], num_img=10, backbone_model=backbone_model, regressor=regressor, 
     yolo_model=model_yolo, yolo_flag=False, yolo_threshold=3, annotations=annotations, 
     plot_flag=True, im_dir=data_path+'images_384_VarV2', enable_gpu=True, 
     model_path= data_path + "pretrainedModels/FamNet_Save1.pth",
     adapt=False, gradient_steps=100, learning_rate=1e-7)